In [1]:
# Import required libraries and dependencies

In [2]:
import keras as krs
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import tensorflow as tf
import sklearn as sk
import pandas as pd
import numpy as np
from pylab import plt, mpl

ModuleNotFoundError: No module named 'keras'

In [ ]:
import simfin as sf

# Set your API-key for downloading data. This key gets the free data.
sf.set_api_key('free')

# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
df = sf.load_shareprices(variant='latest', market='us')

# Print the first rows of the data.
#print(df.head())
df


In [3]:
#import keras as krs
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import tensorflow as tf
import sklearn as sk
import pandas as pd
import numpy as np
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(precision=4, suppress=True)

In [ ]:
def assess_NA(data):
    """
    Returns a pandas dataframe denoting the total number of NA values and the percentage of NA values in each column.
    The column names are noted on the index.
    
    Parameters
    ----------
    data: dataframe
    """
    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)
    
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    # drop rows that don't have any missing data; omit if you want to keep all rows
    df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA

In [5]:
import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline
from pathlib import Path

In [6]:
us_balance_annual = pd.read_csv(
    Path('us-balance-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_balance_annual.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
0,A,45846,USD,2008,Q4,2008-10-31,2008-12-19,2009-12-21,363000000.0,371000000.0,...,NaN,1.330000e+09,2.125000e+09,3.118000e+09,4448000000,7.416000e+09,-7.470000e+09,2.791000e+09,2.559000e+09,7007000000
1,A,45846,USD,2009,Q4,2009-10-31,2009-12-21,2010-12-20,346000000.0,346000000.0,...,1.000000e+06,1.123000e+09,2.904000e+09,3.975000e+09,5098000000,7.558000e+09,-7.627000e+09,2.760000e+09,2.514000e+09,7612000000
2,A,45846,USD,2010,Q4,2010-10-31,2010-12-20,2011-12-16,347000000.0,353000000.0,...,1.501000e+09,3.083000e+09,2.190000e+09,3.377000e+09,6460000000,7.910000e+09,-8.038000e+09,3.444000e+09,3.236000e+09,9696000000
3,A,45846,USD,2011,Q4,2011-10-31,2011-12-16,2012-12-20,347000000.0,355000000.0,...,2.530000e+08,1.837000e+09,1.932000e+09,2.904000e+09,4741000000,8.271000e+09,-8.535000e+09,4.456000e+09,4.316000e+09,9057000000
4,A,45846,USD,2012,Q4,2012-10-31,2012-12-20,2013-12-19,348000000.0,353000000.0,...,2.500000e+08,1.893000e+09,2.112000e+09,3.458000e+09,5351000000,8.495000e+09,-8.707000e+09,5.505000e+09,5.185000e+09,10536000000


In [7]:
supa_balance = us_balance_annual.drop(['SimFinId', 'Currency'], axis=1)

In [8]:
balance_data = supa_balance.groupby(['Ticker','Fiscal Year']).mean()


In [9]:
balance_data

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                            1.429000e+09   
       2009                                            2.493000e+09   
       2010                                            2.649000e+09   
       2011                                            3.527000e+09   
       2012                                            2.351000e+09   
...                                                             ...   
ZYXI   2015                                            8.000000e+03   
       2016                                            2.470000e+05   
       2017                                            5.565000e+06   
       2018                                            1.012800e+07   
       2019                                            1.404000e+07   

                    Accounts & Notes Receivable   Inventories  \
Ticker Fiscal Year                                              
A      2008                         770000000.0  6.460000e+08   
       2009                         595000000.0  5.520000e+08   
       2010                         869000000.0  7.160000e+08   
       2011                         860000000.0  8.980000e+08   
       2012                         923000000.0  1.014000e+09   
...                                         ...           ...   
ZYXI   2015                           2426000.0  3.050000e+05   
       2016                           3028000.0  1.070000e+05   
       2017                           2185000.0  4.230000e+05   
       2018                           2791000.0  8.370000e+05   
       2019                           5833000.0  2.378000e+06   

                    Total Current Assets  Property, Plant & Equipment, Net  \
Ticker Fiscal Year                                                           
A      2008                 3.182000e+09                      8.240000e+08   
       2009                 3.961000e+09                      8.450000e+08   
       2010                 6.169000e+09                      9.800000e+08   
       2011                 5.569000e+09                      1.006000e+09   
       2012                 4.629000e+09                      1.164000e+09   
...                                  ...                               ...   
ZYXI   2015                 2.766000e+06                      8.010000e+05   
       2016                 3.422000e+06                      5.800000e+05   
       2017                 8.371000e+06                      1.880000e+05   
       2018                 1.432400e+07                      8.190000e+05   
       2019                 2.256600e+07                      8.580000e+05   

                    Long Term Investments & Receivables  \
Ticker Fiscal Year                                        
A      2008                                 206000000.0   
       2009                                 163000000.0   
       2010                                 142000000.0   
       2011                                 117000000.0   
       2012                                 109000000.0   
...                                                 ...   
ZYXI   2015                       

In [ ]:
balance_data.isnull().sum()

In [ ]:
assess_NA(balance_data)

In [ ]:
balance_data.info()

In [ ]:
us_cashflow_annual_df = pd.read_csv(
    Path('us-cashflow-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_cashflow_annual_df.head()

In [ ]:
dirty_cash_data = us_cashflow_annual_df.drop(['SimFinId', 'Currency'], axis=1)
cash_flow = dirty_cash_data.groupby(['Ticker','Fiscal Year']).mean()
cash_flow

In [ ]:
cash_flow.isnull().sum()

In [ ]:
assess_NA(cash_flow)

In [ ]:
cash_flow.info()

In [ ]:
cash_flow.stack().head(30)

In [ ]:
cash_flow.describe()

In [18]:
us_income_annual_df = pd.read_csv(
    Path('us-income-annual.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

In [19]:
us_income_annual_df.tail()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
19250,ZYXI,171401,USD,2015,FY,2015-12-31,2016-03-31,2017-04-17,31271234.0,31271234.0,...,-510000.0,-503000.0,-2991000,NaN,-2991000,57000.0,-2934000,NaN,-2911000,-2911000
19251,ZYXI,171401,USD,2016,FY,2016-12-31,2017-04-18,2018-02-28,31271234.0,31271234.0,...,-556000.0,-352000.0,84000,NaN,84000,-15000.0,69000,NaN,69000,69000
19252,ZYXI,171401,USD,2017,FY,2017-12-31,2018-02-28,2018-02-28,32156000.0,33196000.0,...,-1450000.0,-1450000.0,7494000,NaN,7494000,-129000.0,7365000,NaN,7365000,7365000
19253,ZYXI,171401,USD,2018,FY,2018-12-31,2019-02-26,2019-02-26,32503000.0,34043000.0,...,-154000.0,-154000.0,10216000,NaN,10216000,-664000.0,9552000,NaN,9552000,9552000
19254,ZYXI,171401,USD,2019,FY,2019-12-31,2020-02-27,2020-02-27,32439000.0,33963000.0,...,-5000.0,-5000.0,11061000,880000.0,11941000,-2449000.0,9492000,NaN,9492000,9492000


In [20]:
dirty_income = us_income_annual_df.drop(['SimFinId'], axis=1)
income_annual = dirty_income.groupby(['Ticker','Fiscal Year']).mean()
income_annual

Shares (Basic)  Shares (Diluted)       Revenue  \
Ticker Fiscal Year                                                   
A      2008            363000000.0       371000000.0  5.774000e+09   
       2009            346000000.0       346000000.0  4.481000e+09   
       2010            347000000.0       353000000.0  5.444000e+09   
       2011            347000000.0       355000000.0  6.615000e+09   
       2012            348000000.0       353000000.0  6.858000e+09   
...                            ...               ...           ...   
ZYXI   2015             31271234.0        31271234.0  1.164100e+07   
       2016             31271234.0        31271234.0  1.331300e+07   
       2017             32156000.0        33196000.0  2.343200e+07   
       2018             32503000.0        34043000.0  3.191700e+07   
       2019             32439000.0        33963000.0  4.547200e+07   

                    Cost of Revenue  Gross Profit  Operating Expenses  \
Ticker Fiscal Year                                                      
A      2008           -2.578000e+09  3.196000e+09       -2.401000e+09   
       2009           -2.189000e+09  2.292000e+09       -2.245000e+09   
       2010           -2.514000e+09  2.930000e+09       -2.364000e+09   
       2011           -3.086000e+09  3.529000e+09       -2.458000e+09   
       2012           -3.254000e+09  3.604000e+09       -2.485000e+09   
...                             ...           ...                 ...   
ZYXI   2015           -4.937000e+06  6.704000e+06       -9.185000e+06   
       2016           -3.517000e+06  9.796000e+06       -9.156000e+06   
       2017           -4.819000e+06  1.861300e+07       -9.669000e+06   
       2018           -6.038000e+06  2.587900e+07       -1.550900e+07   
       2019           -8.814000e+06  3.665800e+07       -2.559200e+07   

                    Selling, General & Administrative  Research & Development  \
Ticker Fiscal Year                                                              
A      2008                             -1.697000e+09            -704000000.0   
       2009                             -1.603000e+09            -642000000.0   
       2010                             -1.752000e+09            -612000000.0   
       2011                             -1.809000e+09            -649000000.0   
       2012                             -1.817000e+09            -668000000.0   
...                                               ...                     ...   
ZYXI   2015                             -9.185000e+06                     NaN   
       2016                             -9.156000e+06                     NaN   
       2017                             -9.669000e+06                     NaN   
       2018                             -1.550900e+07                     NaN   
       2019                             -2.559200e+07                     NaN   

                    Depreciation & Amortization  Operating Income (Loss)  \
Ticker Fiscal Year                                                         
A      2008                                 NaN                795000000   
       2009                                 NaN                 47000000   
       2010                                 NaN                566000000   
       2011                                 NaN               1071000000   
       2012                                 NaN               1119000000   
...                                         ...                      ...   
ZYXI   2015                                 NaN                 -2481000   
       2016                                 NaN                   640000   
       2017                                 NaN                  8944000   
       2018                                 NaN                 10370000   
       2019                                 NaN                 11066000   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Fiscal Year                        

In [21]:
income_annual.isnull().sum()

Shares (Basic)                                203
Shares (Diluted)                              203
Revenue                                       281
Cost of Revenue                              1838
Gross Profit                                  265
Operating Expenses                             65
Selling, General & Administrative             950
Research & Development                      11650
Depreciation & Amortization                 10699
Operating Income (Loss)                         0
Non-Operating Income (Loss)                   191
Interest Expense, Net                        2156
Pretax Income (Loss), Adj.                      0
Abnormal Gains (Losses)                      7115
Pretax Income (Loss)                            0
Income Tax (Expense) Benefit, Net            1136
Income (Loss) from Continuing Operations        0
Net Extraordinary Gains (Losses)            15139
Net Income                                      0
Net Income (Common)                             0


In [22]:
income_annual.describe()

,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
count,1.905200e+04,1.905200e+04,1.897400e+04,1.741700e+04,1.899000e+04,1.919000e+04,1.830500e+04,7.605000e+03,8.556000e+03,1.925500e+04,1.906400e+04,1.709900e+04,1.925500e+04,1.214000e+04,1.925500e+04,1.811900e+04,1.925500e+04,4.116000e+03,1.925500e+04,1.925500e+04
mean,2.451149e+08,2.496820e+08,6.794764e+09,-4.753004e+09,2.430386e+09,-1.551795e+09,-1.030333e+09,-3.772612e+08,-4.356324e+08,8.509667e+08,-1.147292e+08,-1.266842e+08,7.373757e+08,-1.181633e+08,6.628754e+08,-1.813431e+08,4.939769e+08,4.353438e+07,4.880439e+08,4.851444e+08
std,9.915405e+08,1.011154e+09,2.322183e+10,1.765833e+10,7.667796e+09,4.794611e+09,3.646086e+09,1.278280e+09,1.568861e+09,3.602512e+09,1.275564e+09,4.540450e+08,3.043115e+09,9.193843e+08,3.049554e+09,1.043311e+09,2.297784e+09,6.000995e+08,2.288431e+09,2.287385e+09
min,2.000000e+00,2.000000e+00,-6.741000e+06,-3.946050e+11,-1.280800e+10,-1.087910e+11,-1.087910e+11,-2.601800e+10,-2.921800e+10,-1.347300e+10,-5.912000e+10,-2.575800e+10,-1.489500e+10,-3.426900e+10,-2.869500e+10,-3.653000e+10,-2.071700e+10,-9.559000e+09,-2.235400e+10,-2.280100e+10
25%,2.623300e+07,2.653456e+07,2.733210e+08,-2.892844e+09,1.153652e+08,-1.105516e+09,-7.089900e+08,-1.739250e+08,-2.810698e+08,5.266726e+06,-8.080000e+07,-1.044245e+08,4.955825e+05,-4.085950e+07,-1.680785e+06,-1.109225e+08,-2.005513e+06,-3.838500e+06,-2.121934e+06,-2.614548e+06
50%,6.241700e+07,6.330376e+07,1.329680e+09,-7.544910e+08,5.264455e+08,-3.418300e+08,-1.980000e+08,-5.048900e+07,-7.245050e+07,1.220850e+08,-1.070100e+07,-2.129400e+07,8.890000e+07,-4.184500e+06,7.491900e+07,-1.920000e+07,5.534200e+07,0.000000e+00,5.382300e+07,5.300600e+07
75%,1.765087e+08,1.789880e+08,4.630200e+09,-1.454990e+08,1.733500e+09,-9.037350e+07,-4.665300e+07,-1.388100e+07,-1.516850e+07,5.515500e+08,-3.900000e+04,-8.995000e+05,4.397405e+08,0.000000e+00,4.059500e+08,-2.790000e+05,3.059880e+08,1.341700e+07,3.057000e+08,3.032415e+08
max,2.617490e+10,2.646993e+10,5.239640e+11,1.219000e+09,1.722200e+11,1.110132e+09,0.000000e+00,6.480000e+05,4.962000e+09,1.425170e+11,4.304900e+10,1.537000e+09,1.026960e+11,1.289400e+10,1.026960e+11,3.483100e+10,8.179200e+10,1.591450e+10,8.141700e+10,8.141700e+10


In [23]:
income_annual.stack().head(50)

Ticker  Fiscal Year                                          
A       2008         Shares (Basic)                              3.630000e+08
                     Shares (Diluted)                            3.710000e+08
                     Revenue                                     5.774000e+09
                     Cost of Revenue                            -2.578000e+09
                     Gross Profit                                3.196000e+09
                     Operating Expenses                         -2.401000e+09
                     Selling, General & Administrative          -1.697000e+09
                     Research & Development                     -7.040000e+08
                     Operating Income (Loss)                     7.950000e+08
                     Non-Operating Income (Loss)                 2.000000e+07
                     Interest Expense, Net                      -1.000000e+07
                     Pretax Income (Loss), Adj.                  8.150000e+08
  

In [24]:
assess_NA(income_annual)

NameError: name 'assess_NA' is not defined

In [ ]:
income_annual.info()

In [34]:
us_shareprice_df = pd.read_csv(
    Path('us-shareprices-daily.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)


us_shareprice_df

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,GOOG,18,2014-03-27,568.00,552.92,568.00,558.46,558.46,NaN,13100,336050831.0
1,GOOG,18,2014-03-28,561.20,558.67,566.43,559.99,559.99,NaN,41100,336050831.0
2,GOOG,18,2014-03-31,566.89,556.93,567.00,556.97,556.97,NaN,10800,674462000.0
3,GOOG,18,2014-04-01,558.71,558.71,568.45,567.16,567.16,NaN,7900,674462000.0
4,GOOG,18,2014-04-02,565.11,562.19,604.83,567.00,567.00,NaN,146700,674462000.0
...,...,...,...,...,...,...,...,...,...,...,...
5982671,GOEV,1104613,2020-02-11,10.27,10.27,10.27,10.27,10.27,NaN,2615,8701731.0
5982672,GOEV,1104613,2020-02-12,10.27,10.27,10.27,10.27,10.27,NaN,2,8701731.0
5982673,GOEV,1104613,2020-02-13,10.27,10.27,10.27,10.27,10.27,NaN,326,8701731.0
5982674,GOEV,1104613,2020-02-14,10.38,10.24,10.38,10.28,10.28,NaN,490298,8701731.0


In [38]:
ticker_data = us_shareprice_df.groupby(['Ticker', 'Date']).mean()
ticker_data

SimFinId   Open    Low   High  Close  Adj. Close  Dividend  \
Ticker Date                                                                     
A      2007-01-03     45846  34.99  34.05  35.48  34.30       22.72       NaN   
       2007-01-04     45846  34.30  33.46  34.60  34.41       22.80       NaN   
       2007-01-05     45846  34.30  34.00  34.40  34.09       22.58       NaN   
       2007-01-08     45846  33.98  33.68  34.08  33.97       22.51       NaN   
       2007-01-09     45846  34.08  33.63  34.32  34.01       22.53       NaN   
...                     ...    ...    ...    ...    ...         ...       ...   
ZYXI   2020-02-11    171401  10.02   9.55  10.02   9.88        9.88       NaN   
       2020-02-12    171401   9.92   9.92  10.50  10.32       10.32       NaN   
       2020-02-13    171401  10.24  10.10  10.36  10.26       10.26       NaN   
       2020-02-14    171401  10.20   9.95  10.33  10.16       10.16       NaN   
       2020-02-18    171401  10.15  10.01  10.34  10.18       10.18       NaN   

                    Volume  Shares Outstanding  
Ticker Date                                     
A      2007-01-03  2574600                 NaN  
       2007-01-04  2073700                 NaN  
       2007-01-05  2676600                 NaN  
       2007-01-08  1557200                 NaN  
       2007-01-09  1386200                 NaN  
...                    ...                 ...  
ZYXI   2020-02-11   196899          32791665.0  
       2020-02-12   261188          32791665.0  
       2020-02-13   120219          32791665.0  
       2020-02-14   199752          32791665.0  
       2020-02-18   290273          32791665.0  

[5982676 rows x 9 columns]

In [36]:
ticker_data.isnull().sum()

SimFinId                    0
Open                     8669
Low                      8908
High                     8477
Close                    8682
Adj. Close               6134
Dividend              5937123
Volume                      0
Shares Outstanding    1226448
dtype: int64

In [37]:
assess_NA(billi_bound)

NameError: name 'assess_NA' is not defined

In [ ]:
ticker_data.info


In [ ]:
billi_bound.stack().head(25)

In [33]:
billi_bound.describe()

,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
count,5.982676e+06,5.974007e+06,5.973768e+06,5.974199e+06,5.973994e+06,5.976542e+06,45553.000000,5.982676e+06,4.756228e+06
mean,4.550590e+05,3.456408e+04,3.442578e+04,3.467180e+04,3.453678e+04,3.451902e+04,0.384525,2.133329e+06,2.510479e+08
std,3.018309e+05,1.821466e+06,1.817949e+06,1.823922e+06,1.820574e+06,1.820186e+06,2.584962,1.257065e+07,9.862811e+08
min,1.800000e+01,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,0.000000,0.000000e+00,0.000000e+00
25%,1.852170e+05,1.230000e+01,1.204000e+01,1.254000e+01,1.230000e+01,1.061000e+01,0.110000,1.145708e+05,2.781671e+07
50%,3.782000e+05,2.718000e+01,2.675000e+01,2.759000e+01,2.718000e+01,2.323000e+01,0.210000,4.666000e+05,6.645861e+07
75%,7.003170e+05,5.220000e+01,5.153000e+01,5.287000e+01,5.221000e+01,4.608000e+01,0.390000,1.592700e+06,1.897290e+08
max,1.104613e+06,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,425.500000,6.674913e+09,1.220330e+11


In [ ]:
import simfin as sf

# Set your API-key for downloading data. This key gets the free data.
sf.set_api_key('free')

# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
df = sf.load_income(variant='annual', market='us')

# Print the first rows of the data.
print(df.head(1))
import simfin as sf


In [ ]:
df['Ticker']